<h1 align=center>SP20: SQL AND NOSQL: 12147</h1> 
<h1 align=center>US Parks</h1>
<h3>Project Team</h3>
<li>Huzefa Igatpuriwala - higatpur@iu.edu</li>
<li>Saurabh Swaroop - sswaroop@iu.edu</li>
<li>Vijayakumar Perumalsamy - vperumal@iu.edu </li>


<h5><b>Objective:</b></h5>
To gain more insights from the given NPS dataset. 

In [1]:
import pandas as pd
import numpy as np
import os
from pymongo import MongoClient
import json
from bson.code import Code
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True) 
import plotly.graph_objects as go
import chart_studio
import chart_studio.plotly as py
import plotly.express as px
from bson.son import SON

from plotly.subplots import make_subplots

In [2]:
!jupyter nbextension enable --sys-prefix --py widgetsnbextension 

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [3]:
!jupyter nbextension list

Known nbextensions:
  config dir: /Users/viaandad/ENV3/etc/jupyter/nbconfig
    notebook section
      plotlywidget/extension  enabled 
      - Validating: OK
      jupyter-js-widgets/extension  enabled 
      - Validating: OK


<h3>Data Preprocessing</h3>

The following datasets are mainly used for this visualization.
<ul>
<li>US Parks Visitors Last 10 years.csv</li>
<li>geo_locations.csv</li>
<li>us_states_abbr.csv</li>
</ul>
<br>
Read complete US Parks dataset which includes last 10 years of RV campers, Tent Campers and Recreational Visits data by month/year and process the same.

In [4]:
np_df=pd.read_csv("US Parks Visitors Last 10 years.csv")
print("List of columns available in National Parks dataset:")
print(np_df.columns)

List of columns available in National Parks dataset:
Index(['Park', 'UnitCode', 'ParkType', 'Region', 'State', 'Year', 'Month',
       'RecreationVisits', 'TentCampers', 'RVCampers'],
      dtype='object')


In [5]:
print("National Parks dataset shape:",np_df.shape)
print("National Parks dataset sample:",np_df.head(1))

National Parks dataset shape: (53296, 10)
National Parks dataset sample:                              Park UnitCode                  ParkType  \
0  Abraham Lincoln Birthplace NHP     ABLI  National Historical Park   

       Region State  Year  Month  RecreationVisits  TentCampers  RVCampers  
0  Southeast     KY  2008      1              5829            0          0  


<h4>Geolocations data collection based on each National Park address</h4>
Populated latitude and longitude data from Google API URL for the given park address and API key and updated geo_locations.csv dataset accordingly. Refer the attached <b>Lat-Long-Google.ipynb</b> for more details. 
<br><br>
<b>Google API URL:</b> <a>https://maps.googleapis.com/maps/api/geocode/json?address={}&key={}</a>

Read geo-coordinates data from geo_locations.csv.

In [6]:
np_df_loc=pd.read_csv("geo_locations.csv")
np_df_loc= np_df_loc.rename(columns={"State": "State_Abbr"})
np_df_loc.reset_index()
print("List of columns available in National Park Locations dataset:")
print(np_df_loc.columns)

print("National Park Locations dataset shape:",np_df_loc.shape)
print("National Park Locations dataset sample:")
print(np_df_loc.head(5))

List of columns available in National Park Locations dataset:
Index(['UnitCode', 'ParkName', 'State_Abbr', 'Country', 'Latitude',
       'Longitude'],
      dtype='object')
National Park Locations dataset shape: (379, 6)
National Park Locations dataset sample:
  UnitCode                        ParkName State_Abbr Country   Latitude  \
0     ABLI  Abraham Lincoln Birthplace NHP         KY     USA  37.531540   
1     ACAD                       Acadia NP         ME     USA  44.338556   
2     ADAM                       Adams NHP         MA     USA  42.239235   
3     AFBG        African Burial Ground NM         NY     USA  40.714537   
4     AGFO            Agate Fossil Beds NM         NE     USA  42.425210   

    Longitude  
0  -85.735254  
1  -68.273335  
2  -71.003528  
3  -74.004467  
4 -103.734240  


Read US state full names and abbreviations dataset.

In [7]:
us_states_abbr_df=pd.read_csv("us_states_abbr.csv")
print("List of columns available in US state abbrevation dataset:")
print(us_states_abbr_df.columns)

print("US state abbrevation dataset shape:",us_states_abbr_df.shape)
print("US state abbrevation dataset sample:",us_states_abbr_df.head(5))

#Trim the state_abbr column values
us_states_abbr_df.State_Abbr = us_states_abbr_df.State_Abbr.str.strip()
us_states_abbr_df=us_states_abbr_df.set_index('State_Abbr')

List of columns available in US state abbrevation dataset:
Index(['State', 'State_Abbr'], dtype='object')
US state abbrevation dataset shape: (50, 2)
US state abbrevation dataset sample:          State State_Abbr
0     Alabama          AL
1      Alaska          AK
2     Arizona          AZ
3    Arkansas          AR
4  California          CA


##### Merge both National Park and US state abbreviations dataset to include state full name 


In [8]:
np_df_actual=np_df
np_df_actual= np_df_actual.rename(columns={"State": "State_Abbr"})
np_df_actual.State_Abbr = np_df_actual.State_Abbr.str.strip()
np_df_actual = pd.merge(np_df_actual,us_states_abbr_df, how='left', on='State_Abbr')
np_df_actual.State = np_df_actual.State.str.strip()
np_df_actual.head(10)


,Park,UnitCode,ParkType,Region,State_Abbr,Year,Month,RecreationVisits,TentCampers,RVCampers,State
0,Abraham Lincoln Birthplace NHP,ABLI,National Historical Park,Southeast,KY,2008,1,5829,0,0,Kentucky
1,Abraham Lincoln Birthplace NHP,ABLI,National Historical Park,Southeast,KY,2008,2,6911,0,0,Kentucky
2,Abraham Lincoln Birthplace NHP,ABLI,National Historical Park,Southeast,KY,2008,3,10720,0,0,Kentucky
3,Abraham Lincoln Birthplace NHP,ABLI,National Historical Park,Southeast,KY,2008,4,17829,0,0,Kentucky
4,Abraham Lincoln Birthplace NHP,ABLI,National Historical Park,Southeast,KY,2008,5,26295,0,0,Kentucky
5,Abraham Lincoln Birthplace NHP,ABLI,National Historical Park,Southeast,KY,2008,6,33817,0,0,Kentucky
6,Abraham Lincoln Birthplace NHP,ABLI,National Historical Park,Southeast,KY,2008,7,29225,0,0,Kentucky
7,Abraham Lincoln Birthplace NHP,ABLI,National Historical Park,Southeast,KY,2008,8,22166,0,0,Kentucky
8,Abraham Lincoln Birthplace NHP,ABLI,National Historical Park,Southeast,KY,2008,9,21765,0,0,Kentucky
9,Abraham Lincoln Birthplace NHP,ABLI,National Historical Park,Southeast,KY,2008,10,13747,0,0,Kentucky


In [9]:
np_df_loc_short=np_df_loc[["UnitCode",'Latitude',"Longitude"]]
np_df_final = pd.merge(np_df_actual,np_df_loc_short, how='left', on='UnitCode')
np_df_final.head(10)
np_df_final.shape
np_df_final.to_csv("nps_base_data.csv")

## Import Json documents to Mongodb

In [11]:
def import_content(json_data):
    
    client = MongoClient('localhost', 27017)
    db = client['US_Parks'] 
    collection_name = 'US_Park_Visitor'
    db_cm = db[collection_name]
    if collection_name not in db.list_collection_names():
        db_cm.insert_many(json_data)
    return db,db_cm

def read_csv_to_df(filepath):
    file_res = os.path.join(os.getcwd(), filepath)
    data = pd.read_csv(file_res)
    data_json = json.loads(data.to_json(orient='records'))
    return data_json,data

if __name__ == "__main__":
  filepath = 'nps_base_data.csv'  
  json_data, data = read_csv_to_df(filepath)
  US_Parks_db, US_Parks_collection = import_content(json_data)

## Analysis using Mongodb data

### Verify one document from the collection

In [12]:
US_Parks_collection.find_one()

{'_id': ObjectId('5e9f732c7f107a8522f3d0f1'),
 'Unnamed: 0': 0,
 'Park': 'Abraham Lincoln Birthplace NHP',
 'UnitCode': 'ABLI',
 'ParkType': 'National Historical Park',
 'Region': 'Southeast ',
 'State_Abbr': 'KY',
 'Year': 2008,
 'Month': 1,
 'RecreationVisits': 5829,
 'TentCampers': 0,
 'RVCampers': 0,
 'State': 'Kentucky',
 'Latitude': 37.53154,
 'Longitude': -85.7352542}

### PyMongo aggregation function to group each park type and visualize the count

In [13]:
pipeline = [
             {"$unwind": "$ParkType"},
             {"$group": {"_id": "$ParkType", "count": {"$sum": 1}}},
             {"$sort": SON([("count", -1), ("_id", -1)])}
           ]
y = pd.DataFrame(list(US_Parks_collection.aggregate(pipeline)))    
y = y.rename(columns={"_id": "Park Type"})
fig = px.bar(y, x='Park Type', y='count')
fig.show()

### PyMongo aggregation function to group each park type and visualize coutrywide spread on scatter plot

In [14]:
def sactter_plot_US(data):
    data['text'] = data['_id']+ ''+data['count'].astype(str)
    fig = go.Figure(data=go.Scattergeo(
            locationmode = 'USA-states', 
            lon = data['Longitude'],
            lat = data['Latitude'],
            text = data['text'],
            mode = 'markers',
            marker = dict(
                            size = 8,
                            opacity = 0.99,
                            reversescale = True,
                            autocolorscale = False,
                            symbol = 'square',
                            line = dict(
                                width=1,
                                color='rgba(102, 102, 102)'
                    ),
            colorscale = px.colors.sequential.Inferno,
            cmin = 0,
            color = data['count'],
            cmax = data['count'].max(),
            colorbar_title="Count of recreational, RV and Campers visits"
            )))

    fig.update_layout(
        title = 'Spread of different park types across US)',
        geo = dict(
            scope='usa',
            projection_type='albers usa',
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5
              ),
    )
      
    fig.show()

In [15]:
def scatter_plot(parktype):
    x = pd.DataFrame(list(US_Parks_collection.aggregate( [
                                        {"$match":{'ParkType':parktype}},
                                        {"$group":{
                                                   "_id":"$Park",
                                                   "count": { "$sum": { "$add" : [ 
                                            '$RecreationVisits', '$TentCampers','$RVCampers']}},
                                        }},
                                        {"$sort":{"_id":1}}
                                     ]
                                        )))
    lat_long = pd.DataFrame(list(US_Parks_collection.find(
                                            {'ParkType':'National Historical Park'},
                                            {"_id":0,"Latitude":1,"Longitude":1})
                                 )).drop_duplicates().reset_index(drop=True)
    x['Latitude'] = lat_long['Latitude']
    x['Longitude'] = lat_long['Longitude']
    
    sactter_plot_US(x)

In [16]:
interact(scatter_plot, parktype=data.ParkType.unique());

interactive(children=(Dropdown(description='parktype', options=('National Historical Park', 'National Park', '…

### Pymongo Visualize the number of RecreationVisits, Tent campers and RV campers

In [23]:
def plot_aggregate_metrics(df):
    
    fig = go.Figure()
    fig.update_layout(template='plotly_dark')

    fig.add_trace(go.Scatter(x=df['Month'], 
                             y=df['RecreationVisits'],
                             mode='markers',
                             name='RecreationVisits',
                             line=dict(color='Yellow', width=2)
                            ))
    fig.add_trace(go.Scatter(x=df['Month'], 
                             y=df['TentCampers'],
                             mode='markers',
                             name='TentCampers',
                             line=dict(color='Red', width=2)
                            ))
    fig.add_trace(go.Scatter(x=df['Month'], 
                             y=df['RVCampers'],
                             mode='markers',
                             name='RVCampers',
                             line=dict(color='Green', width=2)
                            ))
    fig.update_layout(title='Monthly count of visitors in each category')
    fig.update_yaxes(title_text='Count of visitors')
    fig.update_xaxes(title_text='Month from January to December')

    return fig

In [24]:
def plot_scatter(park,year):
    data = pd.DataFrame(list(US_Parks_collection.find({"$and":[{'Park':park},{'Year':year.item()}]})))
    return plot_aggregate_metrics(data)

In [25]:
interact(plot_scatter, park=data.Park.unique(),year=data.Year.unique());

interactive(children=(Dropdown(description='park', options=('Abraham Lincoln Birthplace NHP', 'Acadia NP', 'Ad…

### Pymongo Visualize the number of RecreationVisits, Tent campers and RV campers on 10 year average

In [34]:
def plot_line(park):
    recreation = pd.DataFrame(list(US_Parks_collection.aggregate( [
                                        {"$match":{'Park':park}},
                                        {
                                            "$group":{"_id":"$Month",
                                                      "Count":{"$sum":"$RecreationVisits"},
                                                     }
                                        },
                                        {"$sort":{"_id":1}}
                                     ]
                                        )))
    campers = pd.DataFrame(list(US_Parks_collection.aggregate( [
                                        {"$match":{'Park':park}},
                                        {
                                            "$group":{"_id":"$Month",
                                                      "Count":{"$sum":"$TentCampers"},
                                                     }
                                        },
                                        {"$sort":{"_id":1}}
                                     ]
                                        )))
    rvcampers = pd.DataFrame(list(US_Parks_collection.aggregate( [
                                        {"$match":{'Park':park}},
                                        {
                                            "$group":{"_id":"$Month",
                                                      "Count":{"$sum":"$RVCampers"},
                                                     }
                                        },
                                        {"$sort":{"_id":1}}
                                     ]
                                        )))


    
    recreation['Type'] = 'RecreationVisits'
    campers['Type'] = 'TentCampers'
    rvcampers['Type'] = 'RVCampers'
    

    result = pd.concat([recreation, campers, rvcampers],axis=0)
    
    fig = px.line(result, x='_id', y='Count',color='Type', line_group="Type", hover_name="Type")
    
    fig.update_layout(title='Monthly average count of visitors for past 10 years', template='plotly_dark',showlegend=True, )
    
    fig.update_xaxes(title_text='Months from January to December')
    fig.update_yaxes(title_text='Average 10 year count of visitors')
    fig.show()

In [35]:
interact(plot_line, park=data.Park.unique());

interactive(children=(Dropdown(description='park', options=('Abraham Lincoln Birthplace NHP', 'Acadia NP', 'Ad…